In [1]:
import pandas as pd
from neo4j import GraphDatabase


ImportError: cannot import name 'GraphDatabase' from 'neo4j' (c:\Users\Doushka\Downloads\neo4j.py)

In [2]:
def test_connection(driver):
    try:
        # Try to open a session
        with driver.session() as session:
            # Run a simple read query to get the server version
            result = session.run("RETURN 'Connection Successful' AS message")
            for record in result:
                # If the query succeeds, print the result
                print(record['message'])
            return True
    except Exception as e:
        # If an error occurs, print the error message
        print(f"An error occurred: {e}")
        return False
driver = GraphDatabase.driver('neo4j://widb005l.f4.htw-berlin.de:7687', auth=('neo4j', '<passwort festlegen>')) 


# Test the connection
if test_connection(driver):
    print("The connection to the Neo4j database is successful.")
else:
    print("Failed to connect to the Neo4j database.")

NameError: name 'GraphDatabase' is not defined

In [ ]:
def resetDb(driver):
    # Query to delete all nodes and relationships
    delete_query = "MATCH (n) DETACH DELETE n"
    
    # Query to create the desired nodes and relationships
    create_query = """
    CREATE 
      (a1:Admin {name: 'a1'}),
      (a2:Admin {name: 'a2'}),
      (g1:Gruppe {name: 'g1'}),
      (g2:Gruppe {name: 'g2'}),
      (g3:Gruppe {name: 'g3'}),
      (a1)-[:ING]->(g1),
      (a1)-[:ING]->(g3),
      (a2)-[:ING]->(g2),
      (a2)-[:ING]->(g3),
      (u1:Unternehmen {name: 'u1'}),
      (u11:Unternehmen {name: 'u11'}),
      (u111:Unternehmen {name: 'u111'}),
      (u112:Unternehmen {name: 'u112'}),
      (u1111:Unternehmen {name: 'u1111'}),
      (u1112:Unternehmen {name: 'u1112'}),
      (u12:Unternehmen {name: 'u12'}),
      (u121:Unternehmen {name: 'u121'}),
      (u122:Unternehmen {name: 'u122'}),
      (u11)-[:SUB]->(u1),
      (u12)-[:SUB]->(u1),
      (u111)-[:SUB]->(u11),
      (u1111)-[:SUB]->(u111),
      (u1112)-[:SUB]->(u111),
      (u112)-[:SUB]->(u11),
      (u121)-[:SUB]->(u12),
      (u122)-[:SUB]->(u12)
    """
    
    with driver.session() as session:
        # Begin a transaction
        transaction = session.begin_transaction()
        try:
            # Run the delete query within a transaction
            transaction.run(delete_query)
            # Run the create query within a transaction
            transaction.run(create_query)
            # Commit the transaction if both queries succeed
            transaction.commit()
            print("Database has been reset successfully.")
        except Exception as e:
            # Rollback the transaction in case of an error
            transaction.rollback()
            print(f"An error occurred while resetting the database: {e}")
        finally:
            # Close the session
            session.close()
        
def setAuth(gruppe, unternehmen, auth):
    query = """
    MATCH (g:Gruppe {name: $gruppe}), (u:Unternehmen {name: $unternehmen})
    MERGE (g)-[r:AUTH {type: $auth}]->(u)
    RETURN g, r, u
    """
    with driver.session() as session:
        session.run(query, gruppe=gruppe, unternehmen=unternehmen, auth=auth)
      
      
def unsetAuth(gruppe, unternehmen, auth):
    query = """
    MATCH (g:Gruppe {name: $gruppe})-[r:AUTH {type: $auth}]->(u:Unternehmen {name: $unternehmen})
    DELETE r
    """
    with driver.session() as session:
        session.run(query, gruppe=gruppe, unternehmen=unternehmen, auth=auth)
          
def showAuth():
    query = """
    MATCH (a:Admin)-[:ING]->(g:Gruppe)-[r:AUTH]->(u:Unternehmen)
    OPTIONAL MATCH (parent)-[sub_rel:SUB*0..]->(u) // Find all parents (including self)
    OPTIONAL MATCH (u)-[sub_rel2:SUB*0..]->(child) // Find all children (including self)
    WITH a.name AS Admin, r.type AS Auth, 
         collect(DISTINCT parent.name) AS Parents, 
         collect(DISTINCT child.name) AS Children
    RETURN Admin, Auth, Parents, Children
    ORDER BY Admin, Auth // Optional, to order the results by admin and then by auth type
    """
    with driver.session() as session:
        result = session.run(query)
        # Convert the result to a Pandas DataFrame and print it
        data = []
        for record in result:
            data.append({
                "Admin": record["Admin"],
                "Auth": record["Auth"],
                "Parents": record["Parents"],
                "Children": record["Children"]
            })
        df = pd.DataFrame(data)
        print(df)
       

In [ ]:
resetDb()

In [ ]:
# Beispiel 1
setAuth("g3", "u1", "AI")
showAuth()

In [ ]:
# Beispiel 2
setAuth("g1", "u12", "DI")
setAuth("g2", "u11", "DI")
showAuth()

In [ ]:
# Beispiel 3
setAuth("g1", "u111", "DI")
showAuth()

In [ ]:
# Beispiel 4
setAuth("g1", "u1111", "A")
showAuth()

In [ ]:
# Beispiel 5
setAuth("g2", "u12", "D")
showAuth()

In [ ]:
# Beispiel 6
setAuth("g3", "u1", "D")
showAuth()